# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 137 (delta 68), reused 69 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (137/137), 47.34 KiB | 7.89 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_L24O_cv
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

2025-04-29 16:22:35.092175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745943755.413596      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745943755.493690      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from gmrrnet_adhd.models.ShallowConvNet  import ShallowConvNet 

model_name = 'ShallowConvNet'
model_args = {
    'Chans' : 19,
    'Samples' : 512,
    'nb_classes': 2,
    'dropoutRate': 0.5,
    'version': '2018'
}

compile_args = {
    'loss': CategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'optimizer': 'adam',
    'metrics' : ['categorical_accuracy']
}

model = ShallowConvNet(**model_args)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745943773.006542      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745943773.007197      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 19, 512, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast (Cast)                          │ (None, 19, 512, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_1 (Conv2D)                    │ (None, 19, 500, 40)         │             560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_2 (Conv2D)                    │ (None, 1, 500, 40)          │          30,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1, 500, 40)          │             160 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 1, 500, 40)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 1, 67, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1, 67, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 67, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2680)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 2)                   │           5,362 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_activation (Activation)          │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,522 (142.66 KB)

 Trainable params: 36,442 (142.35 KB)

 Non-trainable params: 80 (320.00 B)

# Resultados - Leave 24 Subjects Out +  CV (cada fold se corre 5 veces)

In [5]:
import os

import pickle

with open("/kaggle/input/ieee-tdah-control-database/folds.pkl", "rb") as f:
    folds = pickle.load(f)

In [6]:
results = train_L24O_cv(ShallowConvNet, X, y, sbjs, model_args, compile_args, folds)

Fold 1/5. Test subjects: ['v28p', 'v274', 'v1p', 'v231', 'v22p', 'v29p', 'v206', 'v238', 'v31p', 'v35p', 'v177', 'v200', 'v112', 'v113', 'v48p', 'v140', 'v131', 'v125', 'v55p', 'v143', 'v43p', 'v305', 'v134', 'v114']


I0000 00:00:1745943777.676001      70 service.cc:148] XLA service 0x7ee970004cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745943777.676913      70 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745943777.676935      70 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745943778.031069      70 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745943781.843168      70 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Fold metrics: {'accuracy': 0.9251887202262878, 'recall': 0.9242725467095625, 'precision': 0.9260825625798721, 'kappa': 0.8498487737865739, 'auc': 0.9242725467095625}
Fold 2/5. Test subjects: ['v18p', 'v39p', 'v234', 'v32p', 'v190', 'v6p', 'v254', 'v204', 'v24p', 'v183', 'v246', 'v219', 'v298', 'v41p', 'v47p', 'v308', 'v52p', 'v300', 'v59p', 'v299', 'v302', 'v51p', 'v109', 'v127']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold metrics: {'accuracy': 0.7290167808532715, 'recall': 0.7275627525880022, 'precision': 0.7258563553493189, 'kappa': 0.45307587511661207, 'auc': 0.7275627525880023}
Fold 3/5. Test subjects: ['v215', 'v3p', 'v209', 'v37p', 'v213', 'v15p', 'v284', 'v181', 'v19p', 'v34p', 'v263', 'v244', 'v138', 'v121', 'v46p', 'v54p', 'v120', 'v310', 'v147', 'v50p', 'v56p', 'v107', 'v297', 'v108']


...

Fold metrics: {'accuracy': 0.8223981857299805, 'recall': 0.823881294076046, 'precision': 0.8148763632731268, 'kappa': 0.6362135149976607, 'auc': 0.823881294076046}
Fold 4/5. Test subjects: ['v227', 'v8p', 'v236', 'v14p', 'v196', 'v27p', 'v33p', 'v179', 'v173', 'v10p', 'v265', 'v20p', 'v57p', 'v45p', 'v111', 'v115', 'v53p', 'v118', 'v123', 'v44p', 'v149', 'v303', 'v116', 'v151']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold metrics: {'accuracy': 0.8749259114265442, 'recall': 0.8822194683969868, 'precision': 0.8745586614792993, 'kappa': 0.7496626014744966, 'auc': 0.8822194683969868}
Fold 5/5. Test subjects: ['v279', 'v30p', 'v288', 'v286', 'v250', 'v12p', 'v38p', 'v25p', 'v21p', 'v40p', 'v198', 'v270', 'v117', 'v306', 'v309', 'v110', 'v42p', 'v58p', 'v307', 'v133', 'v304', 'v129', 'v49p', 'v60p']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold metrics: {'accuracy': 0.9308022260665894, 'recall': 0.9316094024406718, 'precision': 0.9319530404999399, 'kappa': 0.8617448578293683, 'auc': 0.9316094024406718}


In [7]:
results

[{'accuracy': 0.9251887202262878,
  'recall': 0.9242725467095625,
  'precision': 0.9260825625798721,
  'kappa': 0.8498487737865739,
  'auc': 0.9242725467095625},
 {'accuracy': 0.7290167808532715,
  'recall': 0.7275627525880022,
  'precision': 0.7258563553493189,
  'kappa': 0.45307587511661207,
  'auc': 0.7275627525880023},
 {'accuracy': 0.8223981857299805,
  'recall': 0.823881294076046,
  'precision': 0.8148763632731268,
  'kappa': 0.6362135149976607,
  'auc': 0.823881294076046},
 {'accuracy': 0.8749259114265442,
  'recall': 0.8822194683969868,
  'precision': 0.8745586614792993,
  'kappa': 0.7496626014744966,
  'auc': 0.8822194683969868},
 {'accuracy': 0.9308022260665894,
  'recall': 0.9316094024406718,
  'precision': 0.9319530404999399,
  'kappa': 0.8617448578293683,
  'auc': 0.9316094024406718}]

In [8]:
import numpy as np

accs = []
for result in results:
    accs.append(result['accuracy'])

np.mean(accs)

0.8564663648605346

In [9]:
import pickle

with open(f'results_L24SO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)